In [1]:
import os
# get GCP key, console -> IAM -> services roles -> key -> add key -> generate json key
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "lens-indexer.json" 

In [2]:
from google.cloud import bigquery

# Initialize the BigQuery client
client = bigquery.Client()

# Define your SQL query, excluding the original 'metadata_json' and adding the transformed one
query = """
SELECT 
    publication_id,
    metadata_snapshot_location_url,
    TO_JSON_STRING(metadata_json) AS metadata_json, 
    metadata_version,
    content,
    language,
    region,
    content_warning,
    main_content_focus,
    tags_vector,
    hide_from_feed,
    is_encrypted,
    app,
    tx_hash,
    timestamp,
    created_at,
    datastream_metadata,
    content_vector
FROM 
    lens-public-data.v2_polygon.publication_metadata 
WHERE 
    `timestamp` >= "2024-08-13 00:00:00 UTC"
"""

# Run the query
query_job = client.query(query)

# Get the result and convert to a Pandas DataFrame
df = query_job.to_dataframe()

# Display the first few rows of the DataFrame
df.shape

(285882, 18)

In [64]:
df['metadata_version'].unique()

array(['3.0.0'], dtype=object)

In [4]:
import json
df["metadata_json"] = df["metadata_json"].apply(lambda x: json.loads(x))

We can know:
1. `main_content_focus` column is the same as `mainContentFocus` insides `metadata_json`
2. `content` column is the same as `content` insides `metadata_json`, which is nullable.
3. `languange` and `region` are derived from `locale` of `metadata_json`. but need to lower or upper all cases
4. `content_warning` is derived from `contentWarning` in `metadata_json`.
5. `tags_vector` is derived from `tags` in `metadata_json`.
6. `hide_from_feed` is `hideFromFeed` in `metadata_json`.
7. `is_encrypted` is based on whether `encryptedWith` exists in `metadata_json`
8. `app` is based on `appId` in `metadata_json`, but need to lower cases
9. `tx_hash` is NOT related to the `txHash` in `metadata_json` for TRANSACTION publication.
10. `created_at` and `timestamp` are similar, and I can't derive it from `metadata_json`
11. `datastream_metadata` not clear, and not relevent
12. `content_vector` is all None

In [6]:
(df['metadata_json'].apply(lambda x:x['lens']['mainContentFocus'])==df['main_content_focus']).all()

True

In [7]:
content = df['metadata_json'].apply(lambda x:x['lens']['content'] if 'content' in x['lens'] else None)

In [10]:
content[df['content']!=content].values

array(['', '', 'So many to choose from ', ..., ' ',
       'ObscuredLanes\n\nWent to hike up a lake in Austria (Drachensee), but the dense fog had other plans - ended up shooting moody fotos all along of everything but the lake 🤷😄\n\n\n1/1\n\n#FotoRaws\n\n\n\n\n',
       'Understand the reality behind crypto and blockchain for a clearer perspective.\n@lens/bajaj @lens/adani @lens/learner1994 @lens/rokky @lens/jocky @lens/simitej '],
      dtype=object)

In [11]:
df[df['content']!=content]['content'].values

array([None, None, 'So many to choose from', ..., None,
       'ObscuredLanes\n\nWent to hike up a lake in Austria (Drachensee), but the dense fog had other plans - ended up shooting moody fotos all along of everything but the lake 🤷😄\n\n\n1/1\n\n#FotoRaws',
       'Understand the reality behind crypto and blockchain for a clearer perspective.\n@lens/bajaj @lens/adani @lens/learner1994 @lens/rokky @lens/jocky @lens/simitej'],
      dtype=object)

In [20]:
locale=df['metadata_json'].apply(lambda x:x['lens']['locale'])

In [23]:
lang = locale.apply(lambda x:x.split('-')[0])
region = locale.apply(lambda x:x.split('-')[1] if len(x.split('-'))>1 else None)

In [42]:
lang.unique()

array(['en', 'zh', 'es', 'et', 'ja', 'ru', 'pt', 'th', 'ko', 'tr', 'vi',
       'id', 'om', 'ku', 'uk', 'ga', 'lv', 'cs', 'it', 'sv', 'pl', 'ro',
       'fa', 'nl', 'el', 'de', 'xh', 'no', 'so', 'is', 'sl', 'tl', 'sw',
       'fr', 'mo', 'eo', 'yo', 'cy', 'lt', 'bn', 'hi', 'fi', 'qu', 'da',
       'jv', 'ts', 'bs', 'lb', 'su', 'hu', 'bg', 'be', 'lg', 'ht', 'af',
       'eu', 'sn', 'ms', 'ca', 'mg', 'ta', 'mt', 'lo', 'sq', 'hr', 'az',
       'tn', 'ar', 'gl', 'uz', 'ha', 'ne', 'hy', 'ur', 'gd', 'la', 'zu',
       'sk', 'tk', 'cv', 'mn', 'sd', 'he', 'kk', 'ml', 'km', 'my', 'mk',
       'sr'], dtype=object)

In [43]:
region.unique()

array([None, 'US', 'GB', 'CN', 'RU', 'DE', 'IN', 'AU'], dtype=object)

In [44]:
df['region'].unique(),df['language'].unique()

(array([None, 'US', 'us', 'GB', 'cn', 'CN', 'ru', 'de', 'RU', 'IN', 'au',
        'gb'], dtype=object),
 array(['en', 'zh', 'es', 'et', 'ja', 'ru', 'pt', 'th', 'ko', 'tr', 'vi',
        'id', 'om', 'ku', 'uk', 'ga', 'lv', 'cs', 'it', 'sv', 'pl', 'ro',
        'fa', 'nl', 'el', 'de', 'xh', 'no', 'so', 'is', 'sl', 'tl', 'sw',
        'fr', 'mo', 'eo', 'yo', 'cy', 'lt', 'bn', 'hi', 'fi', 'qu', 'da',
        'jv', 'ts', 'bs', 'lb', 'su', 'hu', 'bg', 'be', 'lg', 'ht', 'af',
        'eu', 'sn', 'ms', 'ca', 'mg', 'ta', 'mt', 'lo', 'sq', 'hr', 'az',
        'tn', 'ar', 'gl', 'uz', 'ha', 'ne', 'hy', 'ur', 'gd', 'la', 'zu',
        'sk', 'tk', 'cv', 'mn', 'sd', 'he', 'kk', 'ml', 'km', 'my', 'mk',
        'sr'], dtype=object))

In [63]:
df['content_warning'].equals(df['metadata_json'].apply(lambda x:x['lens']['contentWarning'] if 'contentWarning' in x['lens'] else None))

True

In [70]:
df['metadata_json'].apply(lambda x:x['lens']['tags'] if 'tags' in x['lens'] else None)[
    df['metadata_json'].apply(lambda x:x['lens']['tags'] if 'tags' in x['lens'] else None).notna()]

12                              [yup]
17                                 []
18                                 []
19                                 []
20                                 []
                     ...             
285845                             []
285849    [orbcommunitiesphotography]
285850           [orbcommunitieslens]
285864                             []
285865                             []
Name: metadata_json, Length: 45316, dtype: object

In [67]:
df['tags_vector'].unique()

array([None, "'yup'", '', "'text'", "'orbcommunitiesorb'",
       "'orbcommunitieswalk'", "'orbcommunitiesfuckurjpeg'",
       "'orbcommunitiesmemes'", "'orbcommunitiesfood'",
       "'orbcommunitiesbonsai'", "'orbcommunitiesisoh'",
       "'orbcommunitieslenscuba'", "'orbcommunitiesbuy'", "'people'",
       "'channel-buttrfly'", "'orbcommunitiessupermush'",
       "'orbcommunitiessaythanks'", "'orbcommunitieszospours'", "'pets'",
       "'orbcommunitiespets'", "'orbcommunitiesart'",
       "'orbcommunitiesmonniverse'", "'orbcommunitieslens'",
       "'orbcommunitiesmusic'", "'orbcommunitieslensrunningclub'",
       "'orbcommunitiesnyc'", "'podcast'", "'orbcommunitiesallships'",
       "'orbcommunitiessitio'", "'orbcommunitieswac'",
       "'orbcommunitiesartistsbyrefraction'",
       "'orbcommunitiesmentalhealth'", "'orbcommunitiestravel'",
       "'orbcommunitiescollectable'", "'orbcommunitiestouchgrass'",
       "'orbcommunitiesanime'", "'orbcommunitiesentrylevel'",
       "'orbcomm

In [71]:
df['hide_from_feed']

0         False
1         False
2         False
3         False
4         False
          ...  
285877    False
285878    False
285879    False
285880    False
285881    False
Name: hide_from_feed, Length: 285882, dtype: boolean

In [72]:
df['is_encrypted']

0         False
1         False
2         False
3         False
4         False
          ...  
285877    False
285878    False
285879    False
285880    False
285881    False
Name: is_encrypted, Length: 285882, dtype: boolean

In [76]:
encryptedWith = df['metadata_json'].apply(lambda x: True if 'encryptedWith' in x['lens'] else False)

In [84]:
encryptedWith[encryptedWith==True].index

Index([ 81359,  93344,  93429,  98619,  98622, 113708, 126665, 126688, 126861,
       195018, 195024, 207601, 208033, 208106, 208517, 208725, 218837, 219482,
       272701, 272703, 272706, 272709, 272711, 272712, 272714, 272715, 272717,
       272718, 272721, 272723, 272725, 272726, 272727, 272728, 272730, 272732,
       272733, 272735, 272736, 272738, 272745, 272746, 272747, 274168, 274181,
       274191, 274207, 274216, 274255, 274256, 281577, 282840],
      dtype='int64')

In [87]:
(df['is_encrypted']==encryptedWith).all()

True

In [83]:
encryptedWith

array([False,  True])

In [94]:
df['app']

0         phaver
1         phaver
2         phaver
3         phaver
4         phaver
           ...  
285877    phaver
285878    phaver
285879    phaver
285880    phaver
285881    phaver
Name: app, Length: 285882, dtype: object

In [96]:
appId = df['metadata_json'].apply(lambda x: x['lens']['appId'] if 'appId' in x['lens'] else None)
appId

0         phaver
1         phaver
2         phaver
3         phaver
4         phaver
           ...  
285877    phaver
285878    phaver
285879    phaver
285880    phaver
285881    phaver
Name: metadata_json, Length: 285882, dtype: object

In [98]:
appId[appId!=df['app']]

13             Hey
14             Hey
15             Hey
16             Hey
23        Orna.art
            ...   
285846    Buttrfly
285847    Buttrfly
285848         Hey
285862    Buttrfly
285863    Buttrfly
Name: metadata_json, Length: 131899, dtype: object

In [101]:
df[df['app']!=appId]['app']

13             hey
14             hey
15             hey
16             hey
23        orna.art
            ...   
285846    buttrfly
285847    buttrfly
285848         hey
285862    buttrfly
285863    buttrfly
Name: app, Length: 131899, dtype: object

In [111]:
df['tx_hash'].equals(df['metadata_json'].apply(lambda x: x['lens']['txHash'] if 'txHash' in x['lens'] else None))

False

In [109]:
df['tx_hash'].unique()

array([None,
       '0x50c6384d9140682b5e5f882e877d2197a3ed0b83e61f438252bbdd9bc419a668',
       '0x6fe3d687c549120341dd16bce70d87f9846d87e7c5fdff46e3451ae754c73723',
       ...,
       '0x18a24151b3ee633579ac311bf53cfad29d7360e9eb7dcdb9b6eb97d246d07296',
       '0x081dd6d0c9a7b96755f13c052d9e690c8073ee316e8926983dc63b71e15b432a',
       '0xf33612807ae4cb1bcaf74fd77b559051e05229eeaf565b608cd9249c923358fb'],
      dtype=object)

In [112]:
df['metadata_json'].apply(lambda x: x['lens']['txHash'] if 'txHash' in x['lens'] else None).unique()

array([None,
       '0xd9a45506001408a0ba060fd308329c0a23719fbf36f4e6e0d24cfcb60386a274',
       '0xfb21055b8b7c7df00d22d26dab1ce39efd757ebf387c9c2fe40ca10ff9b3f4ed'],
      dtype=object)

In [131]:
txHash=df['metadata_json'].apply(lambda x: x['lens']['txHash'] if 'txHash' in x['lens'] else None)

In [135]:
txHash.unique()

array([None,
       '0xd9a45506001408a0ba060fd308329c0a23719fbf36f4e6e0d24cfcb60386a274',
       '0xfb21055b8b7c7df00d22d26dab1ce39efd757ebf387c9c2fe40ca10ff9b3f4ed'],
      dtype=object)

In [147]:
df[df['tx_hash'].notna()]['metadata_json'].apply(lambda x:x['lens'].keys()).values

array([dict_keys(['appId', 'content', 'id', 'image', 'locale', 'mainContentFocus', 'tags']),
       dict_keys(['appId', 'attachments', 'content', 'hideFromFeed', 'id', 'image', 'locale', 'mainContentFocus', 'title']),
       dict_keys(['appId', 'attachments', 'content', 'hideFromFeed', 'id', 'image', 'locale', 'mainContentFocus', 'title']),
       ...,
       dict_keys(['appId', 'content', 'id', 'locale', 'mainContentFocus', 'tags']),
       dict_keys(['appId', 'content', 'id', 'locale', 'mainContentFocus']),
       dict_keys(['appId', 'content', 'id', 'locale', 'mainContentFocus', 'tags'])],
      dtype=object)

In [149]:
df[df['main_content_focus']=='TRANSACTION']

,publication_id,metadata_snapshot_location_url,metadata_json,metadata_version,content,language,region,content_warning,main_content_focus,tags_vector,hide_from_feed,is_encrypted,app,tx_hash,timestamp,created_at,datastream_metadata,content_vector
193100,0x0788f7-0x02-DA-f8d04a7c,https://statics-v2.lens.dev/public/0x0788f7-0x...,{'$schema': 'https://json-schemas.lens.dev/pub...,3.0.0,Test,en,None,None,TRANSACTION,'chain:1' 'txn:2aRVBgAUCKC6Bg/TCDKcCiNxn7829Ob...,False,False,pob-studio,None,2024-08-21 06:00:36+00:00,2024-08-21 06:00:39.018626+00:00,{'uuid': '589ac02d-b8a4-4ca3-adbf-dfce00000011...,None
193101,0x0788f7-0x02-DA-3ac515ed,https://statics-v2.lens.dev/public/0x0788f7-0x...,{'$schema': 'https://json-schemas.lens.dev/pub...,3.0.0,Test,en,None,None,TRANSACTION,'chain:1' 'txn:2aRVBgAUCKC6Bg/TCDKcCiNxn7829Ob...,False,False,pob-studio,None,2024-08-21 06:12:50+00:00,2024-08-21 06:12:53.383297+00:00,{'uuid': '1992afa4-e68a-47cc-ac1d-07df10100101...,None
193102,0x0788f7-0x02-DA-d784d5ff,https://statics-v2.lens.dev/public/0x0788f7-0x...,{'$schema': 'https://json-schemas.lens.dev/pub...,3.0.0,Test,en,None,None,TRANSACTION,'chain:1' 'txn:2aRVBgAUCKC6Bg/TCDKcCiNxn7829Ob...,False,False,pob-studio,None,2024-08-21 06:15:38+00:00,2024-08-21 06:15:40.974033+00:00,{'uuid': 'a6efa70e-496d-421f-8b73-2c3000101010...,None
193103,0x0788f7-0x02-DA-ac15802c,https://statics-v2.lens.dev/public/0x0788f7-0x...,{'$schema': 'https://json-schemas.lens.dev/pub...,3.0.0,Test,en,None,None,TRANSACTION,'chain:1' 'txn:2aRVBgAUCKC6Bg/TCDKcCiNxn7829Ob...,False,False,pob-studio,None,2024-08-21 06:17:20+00:00,2024-08-21 06:17:22.883425+00:00,{'uuid': 'cee1a9d3-494f-42bf-ad51-5aad00001000...,None
193104,0x0788f7-0x02-DA-dccf0041,https://statics-v2.lens.dev/public/0x0788f7-0x...,{'$schema': 'https://json-schemas.lens.dev/pub...,3.0.0,Test,en,None,None,TRANSACTION,'chain:1' 'txn:2aRVBgAUCKC6Bg/TCDKcCiNxn7829Ob...,False,False,pob-studio,None,2024-08-21 06:26:40+00:00,2024-08-21 06:26:42.906013+00:00,{'uuid': 'a858356f-f3b0-4db1-84ba-0fa111110010...,None
193105,0x0788f7-0x02-DA-e285db39,https://statics-v2.lens.dev/public/0x0788f7-0x...,{'$schema': 'https://json-schemas.lens.dev/pub...,3.0.0,Test,en,None,None,TRANSACTION,'chain:1' 'txn:2aRVBgAUCKC6Bg/TCDKcCiNxn7829Ob...,False,False,pob-studio,None,2024-08-21 06:15:06+00:00,2024-08-21 06:15:07.680407+00:00,{'uuid': 'd72fac5d-5dab-4f24-87d4-466901100110...,None
193106,0x0788f7-0x02-DA-cfede4d6,https://statics-v2.lens.dev/public/0x0788f7-0x...,{'$schema': 'https://json-schemas.lens.dev/pub...,3.0.0,Test,en,None,None,TRANSACTION,'chain:1' 'txn:2aRVBgAUCKC6Bg/TCDKcCiNxn7829Ob...,False,False,pob-studio,None,2024-08-21 06:29:40+00:00,2024-08-21 06:29:43.181826+00:00,{'uuid': '53e77545-8976-4fe1-affc-c44011111110...,None
193107,0x0788f7-0x02-DA-73f29da5,https://statics-v2.lens.dev/public/0x0788f7-0x...,{'$schema': 'https://json-schemas.lens.dev/pub...,3.0.0,Test,en,None,None,TRANSACTION,None,False,False,pob-studio,None,2024-08-19 01:44:30+00:00,2024-08-19 01:44:31.938946+00:00,{'uuid': '137009a7-cc9b-4b3d-a580-ddc811001101...,None
193108,0x0788f7-0x02-DA-a4ac46a7,https://statics-v2.lens.dev/public/0x0788f7-0x...,{'$schema': 'https://json-schemas.lens.dev/pub...,3.0.0,Wut,en,None,None,TRANSACTION,'chain:1' 'txn:2aRVBgAUCKC6Bg/TCDKcCiNxn7829Ob...,False,False,pob-studio,None,2024-08-20 21:08:26+00:00,2024-08-20 21:08:27.577426+00:00,{'uuid': 'd626ad6e-17f1-4451-89d9-fc5501100111...,None
193109,0x0788f7-0x02-DA-fba6a8ee,https://statics-v2.lens.dev/public/0x0788f7-0x...,{'$schema': 'https://json-schemas.lens.dev/pub...,3.0.0,Test[⁽¹⁾](https://example.com),en,None,None,TRANSACTION,'chain:1' 'txn:2aRVBgAUCKC6Bg/TCDKcCiNxn7829Ob...,False,False,pob-studio,None,2024-08-19 20:28:17+00:00,2024-08-19 20:28:20.338186+00:00,{'uuid': '0e68e792-6ed0-41f1-b6f2-621b01100111...,None


In [164]:
(df['created_at']-df['timestamp']).max()

Timedelta('0 days 00:31:31.372575')

0        2024-08-21 14:42:28.521160+00:00
1        2024-08-21 14:41:47.672126+00:00
2        2024-08-21 14:43:46.181902+00:00
3        2024-08-21 14:41:55.535819+00:00
4        2024-08-21 14:41:59.893266+00:00
                       ...               
285877   2024-08-18 02:08:18.960765+00:00
285878   2024-08-18 23:22:30.936463+00:00
285879   2024-08-18 19:02:32.047806+00:00
285880   2024-08-21 04:38:48.590575+00:00
285881   2024-08-13 05:11:23.940150+00:00
Name: created_at, Length: 285882, dtype: datetime64[us, UTC]

In [166]:
df['datastream_metadata'].values

array([{'uuid': '7c5507c4-5d43-4343-82c8-78a311101011', 'source_timestamp': 1724251348526},
       {'uuid': '65eae92a-d594-47e8-bc4f-633801111100', 'source_timestamp': 1724251307676},
       {'uuid': 'b0302ced-5aad-41c9-ae6f-e17b11110101', 'source_timestamp': 1724251426186},
       ...,
       {'uuid': 'cba6089e-af1c-4e96-82a0-dded10011010', 'source_timestamp': 1724007752053},
       {'uuid': '8f5e7aa7-4938-4f06-952d-524900011100', 'source_timestamp': 1724215128610},
       {'uuid': '7c141ebc-61df-46a5-a4e4-adc601100110', 'source_timestamp': 1723525883960}],
      dtype=object)

In [176]:
df['content_vector'].unique()

array([None], dtype=object)